In [ ]:
import numpy as np
import pickle
from keras.optimizers import SGD, Adam
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import csv
import matplotlib.image as im
import matplotlib.pyplot as plt
%matplotlib inline 
from PIL import Image
from PIL import ImageOps


In [ ]:
#----------- Load in the training data -----------------
f = open('newData.csv')
csv_f = csv.reader(f)
Img_file_names = []
for row in csv_f:
  Img_file_names.append(row)
f.close()

y_train = np.zeros((len(Img_file_names)*2))
X_train = np.zeros((len(Img_file_names)*2,55,160,3))

k=0
for i in Img_file_names:
    tmp_img = Image.open(i[0])
    tmp2_img = tmp_img.crop((0,50,320,160))
    tmp2_img.thumbnail((160,55), Image.ANTIALIAS)

    X_train[k,:,:,:] = (np.asarray(tmp2_img)-128.0)/128.0
    y_train[k] = i[1]

    tmp3_img = tmp2_img.transpose(Image.FLIP_LEFT_RIGHT)
    X_train[k+1,:,:,:] = (np.asarray(tmp3_img)-128.0)/128.0
    y_train[k+1] = y_train[k]*-1.0
    k=k+2

img_rows, img_cols = 55, 160
input_shape = (img_rows, img_cols, 3)

In [ ]:
#----------- Model hyper parameters -----------------
nb_epoch = 250
batch_size = 64

model = load_model('previousModel.h5')
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.000001))
hist = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch, validation_split=0.25, shuffle=True)
print(hist.history)

In [ ]:
import json

model.save('newModel.h5') 
json_string = model.to_json()
with open('newModel.json', 'w') as outfile:
    json.dump(json_string, outfile)